In [63]:
import pandas as pd

# 题目1

作为区域仓库货架管理员。需要根据发货需求从不同货架上拣货。已知仓库货架明细表：[仓库，货架编号，库存数量]和仓库拣货数量 270，
请计算每个货架的拣货计划，要求拣货次数最少，使用 Python 或 sql 实现

In [67]:
# 示例数据,可以直接转化为Datafram结构
# 库存表
warehouse_details = [
{"区域": "A", "货架编号": "W1", "库存数量": 100},
{"区域": "A", "货架编号": "W2", "库存数量": 150},
{"区域": "A", "货架编号": "W3", "库存数量": 200},
{"区域": "A", "货架编号": "W4", "库存数量": 40},
{"区域": "A", "货架编号": "W5", "库存数量": 50},
{"区域": "A", "货架编号": "W6", "库存数量": 80},
{"区域": "A", "货架编号": "W7", "库存数量": 10},
]
warehouse_details_df = pd.DataFrame(warehouse_details)

In [68]:
'''
这个问题应该有两种解答的角度
1. 总拣货次数最少
2. 总拣货次数最少的前提下，去每个货架的次数也最少，但是这个太复杂了，我暂时写不出来
'''
# 计算剩余拣货总数量
warehouse_details_df = warehouse_details_df.sort_values(by=['库存数量'],ascending=False).reset_index()
# 设置最大货架的剩余拣货总数量，该货架最后一次拣货批次初始值
warehouse_details_df.loc[0,'剩余拣货总数量'] = warehouse_details_df['库存数量'].sum() - warehouse_details_df.loc[0,'库存数量']
# 考虑最大货架库存数量大于单次拣货上限的情况
warehouse_details_df.loc[0,'该货架最后一次拣货批次'] = int(warehouse_details_df.loc[0,'库存数量']/270) + 1
warehouse_details_df

,index,区域,货架编号,库存数量,剩余拣货总数量,该货架最后一次拣货批次
0,2,A,W3,200,430.0,1.0
1,1,A,W2,150,NaN,NaN
2,0,A,W1,100,NaN,NaN
3,5,A,W6,80,NaN,NaN
4,4,A,W5,50,NaN,NaN
5,3,A,W4,40,NaN,NaN
6,6,A,W7,10,NaN,NaN


In [66]:
for i in range(1,len(warehouse_details_df)):
    # 计算当前行剩余拣货总数量
    warehouse_details_df.loc[i,'剩余拣货总数量'] = warehouse_details_df['库存数量'].sum()- warehouse_details_df.loc[0:i,'库存数量'].sum()
    # 计算该货架最后一次拣货批次
    if warehouse_details_df.loc[0:i,'库存数量'].sum() % 270 != 0:
        warehouse_details_df.loc[i,'该货架最后一次拣货批次'] = int(warehouse_details_df.loc[0:i,'库存数量'].sum()/270) + 1
    else:
        warehouse_details_df.loc[i,'该货架最后一次拣货批次'] = int(warehouse_details_df.loc[0:i,'库存数量'].sum()/270)
warehouse_details_df
        

,index,区域,货架编号,库存数量,剩余拣货总数量,该货架最后一次拣货批次
0,2,A,W3,200,430.0,1.0
1,1,A,W2,150,280.0,2.0
2,0,A,W1,100,180.0,2.0
3,5,A,W6,80,100.0,2.0
4,4,A,W5,50,50.0,3.0
5,3,A,W4,40,10.0,3.0
6,6,A,W7,10,0.0,3.0


## 拣货计划：
按照该货架最后一次拣货批次升序前往货架，每次拣货总数量为270

最后一次拣货批次为i的数量不能装满270时，由比i大的批次中的最小批次所对应货架的货物补充货物数量至270

# 题目2

已知仓库的初始库存为 300, 根据仓库的入库出库表计算 1-7 天的期末库存,
公式: 期末库存 = 上一日期末库存+每日入库-每日出库
使用 Python 或 sql 实现

In [9]:
# 创建每日入库出库的DataFrame
on_the_way_data = {
'天数': [1, 2, 3, 4, 5, 6, 7],
'每日入库': [100, 80, 120, 60, 150, 90, 70],
'每日出库': [50, 70, 90, 40, 100, 80, 60]
}
on_the_way_df = pd.DataFrame(on_the_way_data)
on_the_way_df.loc[0,'期末库存'] = 350
for i in range(1,len(on_the_way_df['期末库存'])):
    on_the_way_df.loc[i,'期末库存'] = on_the_way_df.loc[i-1,'期末库存']+on_the_way_df.loc[i,'每日入库']-on_the_way_df.loc[i,'每日出库']
on_the_way_df


,天数,每日入库,每日出库,期末库存
0,1,100,50,350.0
1,2,80,70,360.0
2,3,120,90,390.0
3,4,60,40,410.0
4,5,150,100,460.0
5,6,90,80,470.0
6,7,70,60,480.0


# 题目3

论述一下误差和偏差的区别?如何消除模型偏差?

误差：由于测量仪器本身的精度影响，导致实际计算结果和理论计算结果之间存在的差异。比如0.3333和1/3之间的差异，和计算机浮点数的计算误差
偏差：由于季节性因素，样本数量，样本本身分布的属性没有被充分考虑的情况下，计算结果和真实结果之间的差异。
比如：
1. 用5年前的季节性因素计算的转化率趋势，在市场环境、用户消费习惯、品牌价值发生变化之后，在最新的季节性因素影响下的转化率趋势会完全不同。
2. 在样本量不完全充足的情况下得出的统计性规律可能因为样本数据波动导致对后续数据的预测失准。
3. 等效替代。比如a产品和b产品仅10%的属性存在差异，使用a产品降价后的清库存效率和b产品可能会存在差异
4. 数据计算错误。。。

如何消除模型偏差（应该是不能完全消除模型偏差的，只能尽可能的减小偏差）：
1. 及时更新模型中受时间因素影响的参数
2. 采用更大样本量计算所得出的规律
3. 对样本口径进行同意
4. 基于行业内知识对等效替代等产经的参数进行修正
5. 设定允许的偏差标准
6. 排查并修正错误数据